# 偏差、方差

在深入了解偏差（bias）、方差（variance）前必须要了解泛化误差、过拟合、欠拟合的概念。如果对这些不够熟悉可以快速浏览一遍另一个速查手册[泛化误差（generalization error）、欠拟合（underfitting）与过拟合（overfitting）](./overfitting-underfitting.ipynb)。



## 直观认识

- 偏差度量了学习算法的期望预测与真实结果的偏离程度，即刻画了学习算法本身的拟合能力。
- 方差度量了学习算法对相同大小的不同训练集进行学习时，学习性能的变化，即刻画了数据扰动所造成的影响。

有一个经典的图：

<img src="./bias-variance.jpg" width="400" />

图中四种情况：

- 偏差低，方差低：偏差低意味着预测点都比较接近靶中心，方差低意味着预测点比较集中
- 偏差低，方差高：偏差低意味着预测点都比较接近靶中心，方差高意味着预测点很分散
- 偏差高，方差低：偏差高意味着预测点都远离靶中心，方差低意味着预测点比较集中
- 偏差高，方差高：偏差高意味着预测点都远离靶中心，方差高意味着预测点很分散

## 数学认识

对测试样本 $\boldsymbol{x}$，令 $y_D$ 表示 $\boldsymbol{x}$ 在训练集中的标记，$y$ 为 $\boldsymbol{x}$ 的真实标记，一般情况下 $y_D = y$，但有可能出现噪声，使得 $y_D \neq y$。$f(\boldsymbol{x};\ D)$ 为训练集 $D$ 上学得模型 $f$ 在 $\boldsymbol{x}$ 上的预测输出。

以回归任务为例，学习算法的期望预测为：

$$ \overline{f} ( \boldsymbol{x} ) = \mathbb{E}_{D} \left[ f \left( \boldsymbol{x};\ D \right) \right] $$

使用样本数相同的不同训练集产生的方差为

$$ var(\boldsymbol{x}) = \mathbb{E}_{D} \left[ \left( f\left( \boldsymbol{x};\ D \right) - \overline{f}(\boldsymbol{x}) \right)^2 \right] $$

噪声为

$$ \varepsilon^2 = \mathbb{E}_{D} \left[ \left( y_D -y \right)^2 \right] $$

期望输出与真实标记的差别称为偏差，即

$$ bias^2(\boldsymbol{x}) = \left( \overline{f}(\boldsymbol{x}) - y \right)^2 $$

为了便于讨论，假定噪声期望为零，即 $ \mathbb{E}_{D} \left[ y_D - y \right] = 0 $。通过简单多项式展开合并，可对算法的期望泛化误差进行分解：

$$ \begin{align*}
E(f; D) &= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - y_{D} \right)^2 \right] \\
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) + \overline{f}(\boldsymbol{x}) - y_{D} \right)^2 \right] \\
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right)^2 \right] + \mathbb{E}_{D} \left[ \left( \overline{f}(\boldsymbol{x}) - y_{D} \right)^2 \right] + 2\mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right) \left( \overline{f}(\boldsymbol{x}) - y_{D} \right) \right] \\
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right)^2 \right] + \mathbb{E}_{D} \left[ \left( \overline{f}(\boldsymbol{x}) - y_{D} \right)^2 \right] + 0 \\
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right)^2 \right] + \mathbb{E}_{D} \left[ \left( \overline{f}(\boldsymbol{x}) - y + y - y_{D} \right)^2 \right] \\ 
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right)^2 \right] + \mathbb{E}_{D} \left[ \left( \overline{f}(\boldsymbol{x}) - y \right)^2 \right] + \mathbb{E}_{D} \left[ \left( y - y_{D} \right)^2 \right] + 2\mathbb{E}_{D} \left[ \left( \overline{f}(\boldsymbol{x}) - y \right) \left( y - y_{D} \right) \right] \\
&= \mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right)^2 \right] + \left( \overline{f}(\boldsymbol{x}) - y \right)^2 + \mathbb{E}_{D} \left[ \left( y - y_{D} \right)^2 \right] + 0
\end{align*}$$

其中，

$$ \begin{align*}
2\mathbb{E}_{D} \left[ \left( f(\boldsymbol{x}; D) - \overline{f}(\boldsymbol{x}) \right) \left( \overline{f}(\boldsymbol{x}) - y_{D} \right) \right] &= 2\mathbb{E}_{D} \big[ f(\boldsymbol{x}; D) - \mathbb{E}_{D} \left[ f(\boldsymbol{x}; D) \right] \big] \cdot \mathbb{E}_{D} \big[ \overline{f}(\boldsymbol{x}) - y_{D} \big] \\
&= 2 \big[ \mathbb{E}_{D} \left[ f(\boldsymbol{x}; D) \right] - \mathbb{E}_{D} \left[ f(\boldsymbol{x}; D) \right] \big] \cdot \mathbb{E}_{D} \big[ \overline{f}(\boldsymbol{x}) - y_{D} \big] \\
&= 0
\end{align*}$$


于是，

$$ E(f; D) = bias^2(\boldsymbol{x}) + var(\boldsymbol{x}) + \varepsilon^2 $$

也就是说，泛化误差可分解为偏差、方差与噪声之和。

偏差度量了学习算法的期望预测与真实结果的偏离程度，即刻画了学习算法本身的拟合能力；方差度量了同样大小的训练集的变动所导致的学习性能的变化，即刻画了数据扰动所造成的影响；噪声则表达了在当前任务上任何学习算法所能到达的期望泛化误差的下界，即刻画了学习问题本身的难度。

偏差-方差分解说明，泛化性能是由学习算法的能力、数据的充分性以及学习任务本身的难度所共同决定的。给定学习任务，为了取得好的泛化性能，则需使偏差较小，即能够充分拟合数据，并且使方差较小，即使得数据扰动产生的影响小。

## 偏差方差窘境

一般来说，偏差与方差是有冲突的，这称为偏差-方差窘境（bias-variance dilemma）。给定学习任务，假定我们能控制学习算法的训练程度，则在训练不足时，学习器的拟合能力不够强，训练数据的扰动不足以使学习器产生显著变化，此时偏差主导了泛化错误率；随着训练程度的加深，学习器的拟合能力逐渐增强，训练数据发生的扰动渐渐能被学习器学习到，方差逐渐主导了泛化错误率；在训练程度充足后，学习器的拟合能力已非常强，训练数据发生的轻微扰动都会导致学习器发生显著变化，若训练数据自身的、非全局的特性被学习器学到了，则将发生过拟合。

<img src="./bias-variance-dilemma.jpeg" width="400" />